In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas de evaluación
from sklearn.metrics import accuracy_score


# Para guardar el modelo
import pickle

In [3]:
df = pd.read_csv('./data/titanic_procesado.csv')

In [4]:
X = df.drop(['Survived'], axis=1)
y = df['Survived']

X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1.0,1.0,0.433152,0.125,0.0,0.368146,NaN,1.0
1,0.0,0.0,0.579431,0.125,0.0,0.615097,C85,0.0
2,1.0,0.0,0.462346,0.000,0.0,0.438286,NaN,1.0
3,0.0,0.0,0.563806,0.125,0.0,0.595112,C123,1.0
4,1.0,1.0,0.563806,0.000,0.0,0.448347,NaN,1.0


In [5]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [6]:
X = pd.get_dummies(X, drop_first=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = X_train.values  # Convertir a NumPy array
y_train = y_train.values  # Convertir a NumPy array
X_test = X_test.values    # Convertir a NumPy array
y_test = y_test.values    # Convertir a NumPy array

In [8]:
# Definir los modelos y sus respectivos hiperparámetros para GridSearch
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'splitter': ['best', 'random'],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3],
            'learning_rate': [0.1, 0.2, 0.3],
            'verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'alpha': [0.1, 1.0, 10.0]
        }
    }
}

# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    # Inicializar GridSearchCV con el modelo y los hiperparámetros
    grid_search = GridSearchCV(
        estimator=info_modelo['modelo'],
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    # Ajustar GridSearchCV con los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Hacer predicciones con el modelo ajustado
    y_pred = grid_search.predict(X_test)
    
    # Calcular la precisión de las predicciones
    precision = accuracy_score(y_test, y_pred)
    
    # Almacenar los resultados del modelo
    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })

    estimadores[nombre] = grid_search.best_estimator_
    
    # Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_

# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(
/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its de

Rendimiento de los modelos de clasificación
                                 Modelo  Precisión
7                  Clasificador XGBoost       0.82
8                     Clasificador LGBM       0.81
3    Clasificador de Bosques Aleatorios       0.80
2     Clasificador de Árbol de Decisión       0.80
1   Clasificador de Vectores de Soporte       0.80
0                   Regresión Logística       0.80
5                 Clasificador AdaBoost       0.80
6      Clasificador K-Nearest Neighbors       0.79
4     Clasificador de Gradient Boosting       0.78
10             Clasificador Naive Bayes       0.77
9                            GaussianNB       0.63
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador XGBoost
Precisión: 0.82


/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/octavios/Documentos/Universidad/ia/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Hacer predicciones con el modelo ajustado
y_pred = grid_search.predict(X_test)

# Calcular la precisión de las predicciones
precision = accuracy_score(y_test, y_pred)

# Almacenar los resultados del modelo
puntajes_modelos.append({
    'Modelo': nombre,
    'Precisión': precision
})

estimadores[nombre] = grid_search.best_estimator_

In [10]:
# Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
if precision > mejor_precision:
    mejor_modelo = nombre
    mejor_precision = precision
    mejor_estimador = grid_search.best_estimator_

In [11]:
# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

Rendimiento de los modelos de clasificación
                                 Modelo  Precisión
7                  Clasificador XGBoost       0.82
8                     Clasificador LGBM       0.81
3    Clasificador de Bosques Aleatorios       0.80
0                   Regresión Logística       0.80
2     Clasificador de Árbol de Decisión       0.80
1   Clasificador de Vectores de Soporte       0.80
5                 Clasificador AdaBoost       0.80
6      Clasificador K-Nearest Neighbors       0.79
4     Clasificador de Gradient Boosting       0.78
10             Clasificador Naive Bayes       0.77
11             Clasificador Naive Bayes       0.77
9                            GaussianNB       0.63
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador XGBoost
Precisión: 0.82


In [17]:
mejor_estimador

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

In [12]:
X_train[0]

array([0.0, 1.0, 0.6158169949398109, 0.0, 0.0, 0.5555992086638706, 1.0,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False

In [13]:
y_train[0]

np.int64(0)

In [14]:
nuevos_datos = np.array([0,1,0.6159084,0,0,0.55547282,1]).reshape(1,-1)

In [15]:
mejor_estimador.predict(nuevos_datos)

ValueError: Feature shape mismatch, expected: 153, got 7

In [16]:
with open('modelo.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)  